# Summary

<p>
This notebook explores the city of Cologne using geospatial or geographic data.<br>
In particular parking meters and hospitals are considered.
</p>

<p>
A version of the notebook is avaible with<br>
<a href="https://nbviewer.jupyter.org/github/RolfChung/geospatial_data_1/blob/d26d4b70ec6f0966059b8b45559756cb767ef2c2/geo2_cologne_2000.ipynb" target="_blank">nbviewer</a><br>
Other than Github it shows folium plots.
</p>

<p>
"Geographic data and information is defined in the ISO/TC 211 series of standards as data and information having an implicit or explicit association with a location relative to Earth (a geographic location or geographic position)."<br>
<a href="https://en.wikipedia.org/wiki/Geographic_data_and_information" target="_blank">Wikipedia</a> 
</p>

<p>
Here this means in practical terms 'longitudes' and 'latitudes' are used. Two coordinate reference systems are mainly applied here: EPSG:4326 and EPSG:3857.
</p>
<p>
"A spatial reference system (SRS) or coordinate reference system (CRS) is a coordinate-based local, regional or global system used to locate geographical entities. A SRS commonly defines a specific map projection, as well as transformations between different SRS."<br>
<a href="https://en.wikipedia.org/wiki/Spatial_reference_system" target="_blank">Wikipedia</a>  
<br>
It is of overall important to apply the right crs for the operation in case.<br>
Otherwise major trouble is ahead.<br>
More on this in the notebook.
</p> 

<p>
    In a Python perspective the <b>Geopandas</b> package is applied here.
</p> 

<p>
"GeoPandas is an open source project to make working with geospatial data in python easier. GeoPandas extends the datatypes used by pandas to allow spatial operations on geometric types. Geometric operations are performed by shapely. Geopandas further depends on fiona for file access and matplotlib for plotting."<br>
<a href="https://geopandas.readthedocs.io/en/latest/index.html" target="_blank">GeoPandas</a>  
</p> 

<p>
I am totally agreeing  with this. In general it makes life easier and is an entry point into the
many geopspatial packages and dependencies behind it.
</p>

<p>
Further topics explored here are:
</p>

<ul>
  <li>Shapefiles</li>
  <li>GeoDataFrames</li>
  <li>GeoJson</li>
  <li>Geospatial Joins</li>
  <li>Geopspatial calculations</li>
  <li>Geopandas plots</li>
  <li>Folium plots</li>
  <li>Choropleth plots</li>
</ul> 

# Import packages

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import os as os
import time
import pprint
import sys
import unidecode


# geographic
import geopandas as geop
import json, requests
from shapely.geometry import Point
from shapely.geometry import Polygon, mapping, shape
from shapely.geometry.multipolygon import MultiPolygon
from shapely import wkt
import folium
import fiona; fiona.supported_drivers



In [ ]:
print("Version Pandas: {}".format(pd.__version__))
print("Version Geopandas: {}".format(geop.__version__))
print("Version Numpy: {}".format(np.__version__))

OS

In [ ]:
import os
# print(os.getcwd())
# print(os.name)
# print(os.listdir())

In [ ]:
import types
def imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            yield val.__name__
list(imports())

In [ ]:
# Seaborn | Style And Color: ttps://www.geeksforgeeks.org/seaborn-style-and-color/
sns.set() 
sns.set_style("darkgrid") 
sns.set_style("ticks", {"xtick.major.size":20, "ytick.major.size":20})
sns.set_theme(style="darkgrid")


In [ ]:
currentwd = os.getcwd()
# print(currentwd)

# Import data

<p>
<b>About the data set:</b><br>
List of parking ticket machines in Cologne, including information on fees and maximum parking time.<br>

<b>Important:</b><br>
The individual parking ticket machines are georeferenced. However, it is possible that with machines that do not have the street and house number combination, this referencing does not match the actual installation location. This data set is therefore constantly updated (at least every six months).<br>
(Translated with Google Translate.)<br>
<a href="https://www.offenedaten-koeln.de/dataset/parkscheinautomaten-koeln" target="_blank">Stadt Köln</a> 
</p> 

<p>
The data is imported with different methods below.
</p> 

In [ ]:
path_1 = 'datasets/PSA_Koeln_2020_0.csv'

In [ ]:
# encoding:  ‘utf-8’
pm_20 = \
pd.read_json("https://offenedaten-koeln.de/api/action/datastore/search.json?resource_id=11667407-a0d4-42ea-95d8-a0c5e40017dc",
             encoding='utf-8')

In [ ]:
sd_1 = open(path_1, mode='r', encoding='utf-8') # Open the file for reading
sd_1_text = sd_1.read() # Read a file’s contents

print(sd_1.closed) #  Check whether file is closed, file is not closed

sd_1.close() # Close file

print(sd_1_text[:430])

In [ ]:
with open(path_1, 'r', encoding='utf-8') as col_2:
    print(col_2.readline())
    col_2.close()

In [ ]:
col_4 = open(path_1, 'r', encoding='utf-8')
col_4_data = col_4.read()
col_4.close()

col_4_lines = col_4_data.split('\n')

col_4_rows=[]

for line in col_4_lines:
    values = line.split(';')
    col_4_rows.append(values)
    
col_4_rows[:1]

In [ ]:
col_4_rows[6]

Alternatively there is a JSON-Api:

In [ ]:
url_pm = \
"https://offenedaten-koeln.de/api/action/datastore/search.json?resource_id=11667407-a0d4-42ea-95d8-a0c5e40017dc"

In [ ]:
pm_20 = \
pd.read_json(url_pm, encoding='utf-8')

In [ ]:
pm_20.head()

This does not work out. This points to a nested dict including meta data.

In [ ]:
pm_data = requests.get(url_pm).json()

print(type(pm_data))

Structure of the nested json dictionary:

In [ ]:
print(pm_data.keys())
print(pm_data['result'].keys())
print(pm_data['result']['records'][:1])

<p>
How to get the records out of the nested dictionary?<br>
Answer:
</p> 

<p>
<b>pandas.json_normalize</b>
pandas.json_normalize(data, record_path=None, meta=None, meta_prefix=None, record_prefix=None, errors='raise', sep='.', max_level=None)<br>
Normalize semi-structured JSON data into a flat table.<br>
<a href="https://pandas.pydata.org/docs/reference/api/pandas.json_normalize.html" target="_blank">pandas.json_normalize</a> 
</p> 




In [ ]:
pm_455 = pd.json_normalize(pm_data, record_path=['result', 'records'])

print(type(pm_455))

In [ ]:
pm_455.info()

In [ ]:
pm_455[['aufstellort', 'geokoordinatenord', 'geokoordinateost']].head()

Normalizing worked out. Despite the data used here is imported as csv.

In [ ]:
cologne_pd = pd.read_csv(path_1, header=0, sep=';', encoding='utf-8')

# Data exploration

In [ ]:
cologne_pd.shape

In [ ]:
type(cologne_pd)

In [ ]:
cologne_pd.columns.to_list()

In [ ]:
pd.DataFrame(zip(cologne_pd.count(), cologne_pd.isna().sum(), 
                 cologne_pd.count() + cologne_pd.isna().sum()), 
            columns=['Not NA', 'NA', 'Len'], index=cologne_pd.columns.to_list())

In [ ]:
cologne_pd.info()

There are 2783 parking meters. Not all geocoordinates for those are provided.

In [ ]:
cologne_pd.head(3)

In [ ]:
cologne_pd.tail(3)

In [ ]:
cologne_pd_555 =cologne_pd.copy()

## Data cleaning

### Type correction

<p>
There are several variables of type string, which should be a number.<br>
For this project the geocoordinates are the most important variables.<br>
So start with those.<br>
The data types of the geocoordinates are strings.<br> 
Those should be floats and converted.<br>
However those are using commas instead of points as decimal seperators. <br>
Before type conversion the commas must be replaced with points.<br>
There are numerous methods to do this, but as we are here in Pandas context, <br>
the choosen method

<p>
In the data the variables
'GeoKoordinateNord' is 'Latitude'<br>
'GeoKoordinateOst' is'Longitude'.
</p> 

<p>
The terms are renamed below.
</p> 


In [ ]:
cologne_pd_111 = \
cologne_pd.rename(columns={'GeoKoordinateNord': 'Latitude', 'GeoKoordinateOst':'Longitude'})

#### Are there outliers in the coordinates?

<p>
The coordinates should all be in simuilar range.<br>
If those are not, this indicates wrong values.
</p> 

<p>
Geographic coordinates of Cologne, Germany in decimal degrees:<br>
Latitude: 50.9333300°<br>
Longitude: 6.9500000°<br>
</p> 

In [ ]:
coord = cologne_pd_111[['Bezirk/Gebiet', 'Latitude', 'Longitude']]

In [ ]:
cgeoost = cologne_pd_111.Longitude.str.replace(',', '.', regex=False).astype(float)

print(cgeoost[:2])
print(type(cgeoost[:2][1]))

In [ ]:
cgenord = cologne_pd_111.Latitude.str.replace(',', '.', regex=False).astype(float)

print(cgenord[:2])
print(type(cgenord[:2][1]))

In [ ]:
coord_2 = coord.assign(Latitude=cgenord, Longitude=cgeoost)
coord_2.head(3)

In [ ]:
coord_2.describe()

In [ ]:
fig, (ax1, ax2) = plt.subplots(2,1, figsize=(12,5))


sns.boxplot(x='Latitude', data=coord_2, ax=ax1)
sns.boxplot(x='Longitude', data=coord_2, ax=ax2)

plt.show()

In [ ]:
coord_2[(coord_2['Latitude']  > 60)  | (coord_2['Latitude']  < 50)]

This two values should be corrected or dropped..

In [ ]:
coord_2[(coord_2['Longitude'] < 1)]

In [ ]:
coord_2[(coord_2['Longitude'] > 10)]

<p>
The longitude "7012081.0" is not possible. 
</p> 


<p>
This indicates a problem with the Longitudes.
The variable is a string and the fraction seperator is in German a comma instead of a point.
A lot of those are from the district "Mühlheim".
Some values are NA.
Before converting the comma is replaced with point, but some values lack erroneously commas,
maybe because of a transcription error.
This corrupts the conversion and the longitudes and must be corrected by
</p> 

#### inserting point at index 2 

<p>
The first step is trying to understand the situation by finding the longitudes without commas
in the string.<br>
The function below is<br>
<b>applying a user defined funtion on the rows of a data frame with the "string.find()" method</b>
</p> 

In [ ]:
def check_comma(row):
        if row.find(",") >= 0:
            value = 1
        elif row.find(",") == -1:
            value = 0
        return value

In [ ]:
cologne_pd_c = cologne_pd_111.copy()

In [ ]:
cologne_pd_c.info()

In [ ]:
cologne_pd_c['Longitude'] = cologne_pd_c['Longitude'].astype(str)

In [ ]:
cologne_pd_c['commas'] = cologne_pd_c['Longitude'].apply(check_comma)
cologne_pd_c_2 = cologne_pd_c[['Bezirk/Gebiet', 'Longitude', 'commas']]

Printing longitudes with commas.

In [ ]:
print(cologne_pd_c_2.shape)
cologne_pd_c_2.head()

Printing longitudes without commas.

In [ ]:
not_commas = cologne_pd_c_2[cologne_pd_c_2.commas==0]
print(not_commas.shape)
not_commas.head()

In [ ]:
not_commas.info()

Print the entire data frame with the context options manager.<br>
This shows the longitudes without commas.

In [ ]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(not_commas)

#### Checking the latitudes

<p>
The expectation is that there are only nan values, but none latitudes without commas,
based on the describe-statistics.
</p> 

In [ ]:
cologne_pd_c.Latitude.isna().sum()

In [ ]:
cologne_pd_c.dtypes

In [ ]:
cologne_pd_c['Latitude'].head()
print(type(cologne_pd_c['Latitude'][100]))

In [ ]:
lat_122 = cologne_pd_c['Latitude'].astype(str).apply(check_comma)

In [ ]:
lat_122.value_counts()

In [ ]:
cologne_pd_c_33 = cologne_pd_c.copy()
cologne_pd_c_33['lat_checked'] = lat_122
c_lat = cologne_pd_c_33[['Bezirk/Gebiet', 'Latitude', 'lat_checked']]
c_lat[c_lat.lat_checked == 0][:5]

In [ ]:
c_lat.loc[c_lat.lat_checked == 0, 'Latitude'].isna().sum()

The expectation is confirmed.

Now we want to add commas only to the longitude values without commas and not the nan's either.

<b> Example: Find and insert a String into a String in Python with the "string.find()" method</b>

In [ ]:
# take 
mgeo_str=cologne_pd_111.loc[2126,'Longitude']
print(mgeo_str)
print(type(mgeo_str))
print(mgeo_str[0])
print(len(mgeo_str))
print(mgeo_str[0] + "," + mgeo_str[1:len(mgeo_str)])

<p>
The function below is<br>
<b>applying a user defined funtion on the rows of a data frame 
with the "string.find()" method and adds a comma to longitude values.</b>
</p> 

In [ ]:
def add_comma(row):
        if row.find(",") >= 0:
            value = row
            
        elif 'nan' in row:
            value = 'nan'
            
        elif row.find(",") == -1:
            value = row[0] + "," + row[1:len(row)]
        return value

In [ ]:
cologne_pd_c['add_commas'] = cologne_pd_c['Longitude'].apply(add_comma)

In [ ]:
print(cologne_pd_c.shape)
cologne_pd_c.columns

In [ ]:
cologne_pd_c_3 = cologne_pd_c[['Bezirk/Gebiet', 'Longitude', 'commas', 'add_commas']]

In [ ]:
cologne_pd_c_3.head()

In [ ]:
cologne_pd_c_3[cologne_pd_c_3.commas == 0]

In [ ]:
cologne_pd_c_777 = cologne_pd_c.copy()

In [ ]:
cologne_pd_c_777.drop(['Longitude', 'commas'], axis=1, inplace=True)

In [ ]:
cologne_pd_c_777.rename(columns={'add_commas': 'Longitudes', 'Latitude':'Latitudes'}, inplace=True)

In [ ]:
cologne_pd_c_777.columns.to_list()

In [ ]:
long_777 = cologne_pd_c_777.Longitudes.str.replace(',', '.', regex=False).astype(float)

In [ ]:
print(len(long_777))
print(long_777[:4])
print(type(long_777[3]))

In [ ]:
cologne_pd_c_777['Longitudes'] = long_777

In [ ]:
lat_777 = cologne_pd_c_777.Latitudes.str.replace(',', '.', regex=False).astype(float)

In [ ]:
cologne_pd_c_777['Latitudes'] = lat_777

In [ ]:
cologne_pd_c_777.dtypes

In [ ]:
colo_df = cologne_pd_c_777.copy()

In [ ]:
# check if col there with helper function
def col_exist(col, cols):
    if col in cols:
        return ('{col} {text}'.format(col=col, text = 'does exist.'))
    else:
        return ('{col} {text}'.format(col=col, text = 'does not exist.'))

In [ ]:
print(col_exist(col='Longitudes', cols=colo_df.columns))
print(col_exist(col='Latitudes', cols=colo_df.columns))

In [ ]:
drops_123 = coord_2[(coord_2['Latitude']  > 60)  | (coord_2['Latitude']  < 50.8)].index.to_list()
drops_123

In [ ]:
colo_df = colo_df.drop(index=drops_123)

In [ ]:
colo_df.describe()

<p>
The max longitude of 9.955520 is suspicious.
</p> 


In [ ]:
colo_df[colo_df.Longitudes==colo_df.Longitudes.max()]

<p>
The geo coordinates are pointig to 36214 Nentershausen, Germany in Hesse.<br>
The geo coordinates of Richartzstr. 6, Cologne are 50.939500, 6.955570.
</p> 

In [ ]:
colo_df[colo_df.Longitudes > 7.5]

In [ ]:
colo_df.loc[549, 'Longitudes'] = 6.955570
colo_df.loc[1081, 'Longitudes'] = 6.954040

# Check
print(colo_df.loc[549, 'Longitudes'])
print(colo_df.loc[1081, 'Longitudes'])


In [ ]:
print(colo_df.Longitudes.max())

#### Variable: 'Gebühr je 20 Minuten'

In [ ]:
print(cologne_pd['Gebuehr je 15 Minuten'].head())
print(type(cologne_pd.loc[1, 'Gebuehr je 15 Minuten']))

In [ ]:
# str.strip()  function is used to remove or strip the leading 
# and trailing space of the column in pandas dataframe.

gebuehren = \
cols=colo_df['Gebuehr je 15 Minuten'].str.replace(',', '.').str.replace('€', "").str.strip().\
astype(float)

print(gebuehren.head())
print(type(gebuehren[1]))

In [ ]:
colo_df['Gebühr je 15 Minuten'] = gebuehren
colo_df['Gebühr je 15 Minuten'].head()

#### Variable: Stellplätze

In [ ]:
cologne_pd = colo_df.assign(Stellplätze = cologne_pd['Stellplätze'].\
                            fillna(np.median(0).astype(int)))

cologne_pd['Stellplätze'].isna().sum()

#### Variable: Roter Punkt
<p>
is unnecessary here and dropped.
</p> 


In [ ]:
colo_df.drop('Roter Punkt', axis=1, inplace=True)


In [ ]:
colo_df.columns.to_list()

In [ ]:
colo_df.columns = colo_df.columns.str.replace(" ", "_")

In [ ]:
colo_df.columns.to_list()

### Group by

In [ ]:
bezirk_gb = colo_df.groupby('Bezirk/Gebiet')['PSA-Nr.'].count()
print(type(bezirk_gb))
print(len(bezirk_gb))
print(bezirk_gb)

In [ ]:
bezirk_gb.sort_values(ascending=False).plot(kind='bar',
                                            edgecolor='black', linewidth=3,
                                            color=['cyan', 'magenta', 'red'],
                                            figsize=(14,4), 
                                            title='Parking meter by city district')
plt.grid()
plt.show()

In [ ]:
abschnitt_gb = colo_df.groupby('Abschnitt/-bis')['Stellplätze'].sum()
abschnitt_gb_df = pd.DataFrame(abschnitt_gb)
abschnitt_gb_df.info()

In [ ]:
abschnitt_gb_df.head()

In [ ]:
abschnitt_gb_df.tail()

In [ ]:
top_ten_stellplätze = abschnitt_gb.sort_values(ascending=False)[:10]
print(top_ten_stellplätze)

In [ ]:
top_ten_stellplätze.plot(kind='bar', edgecolor='black', linewidth=3, figsize=(10,4),
                         color=['black', 'green', 'red'], legend=False,
                         title='Sum of parking spaces by section')

plt.rc('axes', titlesize=20)

font = {'family' : 'arial',
        'weight' : 'bold',
        'size'   : 18}
plt.rc('font', **font)  # pass in the font dict as kwargs
 
plt.grid()
plt.show()

# Geopspatial data exploration

<p>
In the data the variables
'GeoKoordinateNord' is 'Latitude'<br>
'GeoKoordinateOst' is'Longitude'.
</p> 

<p>
The terms are renamed below.
</p> 


In [ ]:
colo_df_2 = colo_df.copy()
print(colo_df_2.shape)
print(type(colo_df_2))

In [ ]:
colo_df_2 = colo_df_2.dropna(subset=['Latitudes', 'Longitudes'])
print(colo_df_2.shape)

In [ ]:
Latitudes = colo_df_2['Latitudes']
Longitudes = colo_df_2['Longitudes']

In [ ]:
geocoord_111 = colo_df_2[['Latitudes', 'Longitudes']]
geocoord_111.info()

In [ ]:
geocoord_111.describe()

In [ ]:
geocoord_111.dtypes

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3,1, figsize=(9,25))

sns.scatterplot(x='Latitudes', y='Longitudes', 
                data=colo_df_2, color='r', edgecolor='black', ax=ax1)
ax1.set_title('Locations of parking meters in Cologne, Germany in terms of longitude and latitude', 
              fontsize=12)
# ax1.set_xlim(left=50.8, right=51)
# ax1.set_ylim(bottom=6.8, top=7.15)


sns.regplot(x='Latitudes', y='Longitudes', 
            data=colo_df_2, color='g', ax=ax2, scatter=True,
            scatter_kws={'s': 40, 'alpha': 0.9, 'color': 'red',
                         'edgecolor':'black', 'linewidth':0.4, 'alpha':0.8})
ax2.set_title('Regression plot', fontsize=14)
# ax2.set_xlim(left=50.8, right=51)
# ax2.set_ylim(bottom=6.8, top=7.15)


sns.scatterplot(x='Latitudes', y='Longitudes',
                data=colo_df_2, color='r', edgecolor='black', ax=ax3,
                hue='Bezirk/Gebiet', legend='brief')
ax3.set_title('Parking meters colored by district', fontsize=14)
ax3.set_xlim(left=50.8, right=51)
ax3.set_ylim(bottom=6.8, top=7.15)
ax3.legend(bbox_to_anchor=(1,1), loc="upper left", ncol=2)


for ax in fig.axes:
    plt.sca(ax)
    plt.xticks(rotation=90)

plt.subplots_adjust(hspace=0.2)
plt.show()

## Constructing a geopandas data frame from the parking meters geocoordinates

<p>
A geopandas data frame needs a "geometry" column.<br>
This is a special geography format.<br>
The schools data frame is a normal pandas data frame.<br>
It stores lat and long, but not in the geometry format.<br>
The shapely package offers options to create geographic formats like points, lines, and polygons.<br>
The school locations are points.<br>
Below the lats and longs are turned into shape points.
</p> 

In [ ]:
# from shapely.geometry import Point
# colo_df_2.columns

colo_df_2['geometry'] = colo_df_2.apply(lambda x: Point((x.Longitudes, x.Latitudes)), axis=1)

In [ ]:
colo_df_2[['PSA-Nr.', 'Latitudes', 'Longitudes', 'geometry']].head()

#### Converting the data frame to a geopandas df.

In [ ]:
pm_geopd = geop.GeoDataFrame(colo_df_2, crs="EPSG:4326", geometry='geometry')

print(type(pm_geopd))
print(pm_geopd.crs)
# print(pm_geopd.info())
# pm_geopd

## Shapefiles of cologone

<p>
used with<br>
<b>GeoPandas 0.9.0</b><br>
"GeoPandas is an open source project to make working with geospatial data in python easier. GeoPandas extends the datatypes used by pandas to allow spatial operations on geometric types. Geometric operations are performed by shapely. Geopandas further depends on fiona for file access and matplotlib for plotting."<br>
<a href="https://geopandas.org/" target="_blank">GeoPandas</a> 

</p> 


In [ ]:
sf_1 = \
geop.read_file('datasets/Cologne_Stadtteile/Stadtteil.shp', 
               delimiter=',')

In [ ]:
sf_1  = sf_1.to_crs('EPSG:4326')

In [ ]:
sf_1.columns

In [ ]:
sf_1.geometry.crs

In [ ]:
sf_1.geometry[:1]

In [ ]:
print(type(sf_1))
print(sf_1.dtypes)

In [ ]:
print(sf_1.shape)

In [ ]:
print(sf_1.columns)

In [ ]:
print(sf_1.index)

In [ ]:
nuniquue_col = [sf_1[i].nunique() for i in sf_1]
nuniquue_col

In [ ]:
print(pd.DataFrame(zip(sf_1.count(), sf_1.isna().sum(), nuniquue_col), 
                   columns=['n_values', 'nas', 'unique']))

In [ ]:
print(sf_1.info())

There are 86 eight districts in Cologne.

In [ ]:
sf_1.head()

In [ ]:
print(type(sf_1))
print(sf_1.geometry.crs)
sf_1.crs

In [ ]:
# sf_33 = sf_1.to_crs('EPSG:4328')
# sf_33 

Exploring the shape of Cologne visually.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(8,5))
sf_1.plot(color='yellow', ax=ax1, edgecolor='black')
ax1.set_title('Shape')
sf_1.plot(cmap='Set3', ax=ax2)
ax2.set_title('Colormap')
plt.show()

#### Boundaries of geometries

geopandas.GeoSeries.boundary

Returns a GeoSeries of lower dimensional objects representing each geometries’s 
set-theoretic boundary.

In [ ]:
sf1_boundaries = sf_1.geometry.boundary
print(sf1_boundaries[:3])
sf1_boundaries.plot(color='seagreen')
plt.show()

geopandas.GeoSeries.exterior

Returns a GeoSeries of LinearRings representing the outer boundary of each polygon in the GeoSeries.

In [ ]:
sf1_exterior_boundaries = sf_1.geometry.exterior
print(sf1_exterior_boundaries[:3])
sf1_exterior_boundaries.plot()
plt.show()

<p>
Extracting the outer boundaries of a geometry series.
</p>

<strong>GeoSeries.unary_union</strong>

<p>
Return a geometry containing the union of all geometries in the GeoSeries.
</p>


In [ ]:
sf_1_unary = sf_1.geometry.unary_union
sf_1_unary

Further below this shape is used to caculate the center or centroid of Cologne.

In [ ]:
# for i in pd.Series(sf_1.iloc[0,4]):
    # print(i)

#### Coordinate Reference Systems
<p>
"The Coordinate Reference System (CRS) is important because the geometric shapes in a GeoSeries or GeoDataFrame object are simply a collection of coordinates in an arbitrary space. A CRS tells Python how those coordinates relate to places on the Earth."<br>
<a href="https://geopandas.org/docs/user_guide/projections.html" target="_blank">geopandas</a> 
</p> 

<p>
The geo coordinates of the parking meters are in CRS "WGS 84".<br>
The geo coordinates of the shape file are in "WGS 84 / UTM zone 32N".<br>
Latest when plotted together this raises a conflict.
</p> 

In [ ]:
sf_1.geometry.crs

In [ ]:
sf_2 = sf_1.copy()

print(type(sf_2))
print(sf_2.columns.to_list())
print(sf_2.head(5))

In [ ]:
sf_2=sf_2.to_crs('EPSG:4326') 
sf_2.crs

In [ ]:
# for i in pd.Series(sf_2.iloc[0,4]):
    # print(i)

Look into variables

In [ ]:
colog_districts =sf_2.STT_NAME.unique().tolist()
colog_districts[:5]

#### Shape of district: Flittard

In [ ]:
sf_2.loc[0, 'geometry']

#### Shape of district: Dellbrück

In [ ]:
sf_2.loc[2, 'geometry']

Plot the shape of a special district by subsetting with loc.

In [ ]:
sf_2.loc[sf_2.STT_NAME=='Rodenkirchen']

In [ ]:
sf_2.loc[sf_2.STT_NAME=='Rodenkirchen'].plot(edgecolor='black', linewidth=4,
                                             color='yellowgreen')
plt.show()

In [ ]:
sf_2.loc[sf_2.STT_NAME == 'Langel']

In [ ]:
sf_2.loc[sf_2.STT_NAME == 'Langel'].plot(edgecolor='black', linewidth=4,
                                         color='yellowgreen')
plt.show()

#### Shape of cologne

In [ ]:
fig, (ax1, ax2) = plt.subplots(2,1, figsize=(15,15))

sf_2.plot(color='gold', edgecolor='black', linewidth=1, ax=ax1)
ax1.set_title("Shape of Cologne")

sf_2.plot(edgecolor='black', linewidth=1, ax=ax2, legend=True, column='STT_NAME',
          legend_kwds={ 'bbox_to_anchor':(1, 1), 'ncol':3, 'loc':'upper left'})
ax2.set_title("Shape of Cologne colored by district")


plt.show()

### Working with attribute methods of Geoseries geometry data
<p>
starting by:<br> 
</p> 

#### Calculating the areas of Cologne
Examples

In [ ]:
sf_2.loc[sf_2.STT_NAME=='Flittard'].plot(edgecolor='black', linewidth=4,
                                             color='yellowgreen')
plt.title('Flittard')
plt.show()

In [ ]:
area_flit = sf_1.geometry[0].area
print("Area of {name}: {area}".format(name= sf_1.loc[0, 'STT_NAME'], 
                                      area= area_flit))
print("Area of {name} by 'SHAPE_AREA': {area}".format(name= sf_1.loc[0, 'STT_NAME'], 
                                                      area= sf_1.loc[0, 'SHAPE_AREA']))

In [ ]:
sf_2.loc[sf_2.STT_NAME=='Altstadt/Süd'].plot(edgecolor='black', linewidth=4,
                                             color='yellowgreen')
plt.title('Altstadt/Süd')
plt.show()

In [ ]:
area_alt = sf_1.geometry[10].area
print("Area of {name}: {area}".format(name= sf_1.loc[10, 'STT_NAME'], area= area_alt))
print("Area of {name} by 'SHAPE_AREA': {area}".format(name= sf_1.loc[10, 'STT_NAME'], 
                                                      area= sf_1.loc[10, 'SHAPE_AREA']))

In [ ]:
all_areas = sf_1.geometry.area 

What crs is applied here?

In [ ]:
sf_1.geometry.crs

<p>
The WGS 84 -- WGS84 - World Geodetic System 1984, used in GPS uses the CRS-norm: EPSG:4326.<br>
<a href="https://epsg.io/4326" target="_blank">epsg</a> 
</p> 

<p>
Above the areas are calculated in decimal places according to the EPSG: 4326 crs.<br>
This is not human friendly.<br>
A better understanding is enabled by calculating m^2 and km^2.<br>
This only works only for small distances.<br>
Otherwise you have to correct for the fact that the
earth is a sphere and not a disc ;-)<br>
More on <a href="https://gis.stackexchange.com/questions/242545/how-can-epsg3857-be-in-meters" target="_blank">'How can EPSG:3857 be in meters?'</a> here.
</p>

<p>
Using epsg=3857 for meters as a csr returns m^2.
</p> 

<p>
<b>Convert m squared to km squared:</b><br>
"How many m squared in 1 km squared? The answer is 1.000.000.
We assume you are converting between square metre and square kilometre.
You can view more details on each measurement unit:
m squared or km squared
The SI derived unit for area is the square meter.
1 square meter is equal to 1.0E-6 km squared."<br>
<a href="https://www.convertunits.com/from/m+squared/to/km+squared" target="_blank">convertunits.com</a> 
</p> 


In [ ]:
colog_m3857 = sf_1.to_crs(epsg=3857)
colog_m3857.crs

In [ ]:
sf_1[['STT_NAME', 'geometry']].head(2)

In [ ]:
colog_m3857[['STT_NAME', 'geometry']].head(2)

The POLYGON values have changed.

In [ ]:
colog_m_areas = colog_m3857.geometry.area
colog_m_areas[:2]

Conversion of meters to km^2.

In [ ]:
conversion_m_km = 1000000
colog_areas_km = colog_m_areas / conversion_m_km
colog_areas_km = [round(v,4) for v in colog_areas_km]

print(type(colog_areas_km))
print(type(colog_areas_km[0]))

Conversion of meters km^2 to miles.

In [ ]:
conversion_km_miles = 0.3861
colog_areas_miles = [v*conversion_km_miles for v in colog_areas_km]
colog_areas_miles = [round(float(v),4) for v in colog_areas_miles]
# colog_areas_miles

Stylish data frame<br>
<a href="https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html" 
target="_blank">Styling</a> 

In [ ]:
styles = [
    dict(selector="th", props=[("font-size", "110%"),
                               ("text-align", "center")]),
    dict(selector="caption", props=[("caption-side", "top"), ("font-size", "150%")])]

In [ ]:
colog_areas_df = \
pd.DataFrame(zip(colog_m_areas, colog_areas_km, colog_areas_miles, all_areas), 
             columns=['meters', 'km', 'miles', "WGS84"] )
colog_areas_df_style = \
colog_areas_df.style.set_table_styles(styles).set_caption("Size of Cologne areas")

In [ ]:
colog_areas_df_style

#### GeoSeries Centroids
<p>
centers are derived from geometry variables.<br>
Geometry variables are shapes or areas consisting of line or polygons.<br>
Centroids are point in the centers of areas.
</p> 

In [ ]:
# sf_1.info()

In [ ]:
sf_geo = sf_1.geometry
print(type(sf_geo ))

sf_geo_buffer = sf_geo.boundary
sf_geo_buffer.plot()

In [ ]:
sf_geo 

In [ ]:
sf_1.crs

In [ ]:
sf_2.crs

In [ ]:
sf_566_m = sf_2.to_crs("EPSG:3857")

<p>
This simple explicit conversion above was necessary on ground of an error below.<br>
The distances were calculated with different measures. This is corrected now.<br>
The Point values were deviating extremly.<br>
Why was this the case?<br>
It was clear that a pseudo mercator projection in meter was required.<br>
It seems that the whole GeoDataFrame must be converted to "EPSG:3857".<br>
Before only a subset (e.g. Ensen was converted).<br>
Figuring this out was time consuming.
</p> 


Determining the center of district Ensen with the centroid-method implemented by Geopandas.

In [ ]:
print(sf_566_m.STT_NAME[25])
ensen_c = sf_566_m.geometry.centroid[25]
print(ensen_c )
print(type(ensen_c ))

In [ ]:
ensen_cgs = geop.GeoSeries(ensen_c, crs="EPSG:3857")
print(type(ensen_cgs))
print(ensen_cgs.crs)


In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,5))

sf_566_m.loc[sf_566_m.STT_NAME == 'Ensen'].plot(edgecolor='black', linewidth=2,
                                                color= 'yellowgreen',ax=ax)
ensen_cgs.plot(ax=ax, edgecolor='black', linewidth=3, cmap='gist_rainbow')

for ax in fig.axes:
    plt.sca(ax)
    plt.xticks(rotation=90)

plt.title('Centroid of Ensen')
plt.show()

Determining the center of district Grengel with the centroid-method implemented by Geopandas.

In [ ]:
print(sf_566_m.STT_NAME[63])
grengel_c = sf_566_m.geometry.centroid[63]
grengel_cgs = geop.GeoSeries(grengel_c, crs='EPSG:3857')

print(grengel_c)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,5))

sf_566_m.loc[sf_566_m.STT_NAME == 'Grengel'].plot(edgecolor='black', linewidth=2,
                                                color= 'yellowgreen',ax=ax)
grengel_cgs.plot(ax=ax, edgecolor='black', linewidth=3, cmap='gist_rainbow')

for ax in fig.axes:
    plt.sca(ax)
    plt.xticks(rotation=90)

plt.title('Centroid of Grengel')
plt.show()

<strong>
Determining the center of Cologne at a whole with the centroid-method implemented by Geopandas.
</strong>
<br>
<p>
Problem: the shape file of Cologne is not given here.<br>
The shape is extracted from the geometries.<br>
As a first step the shape of Cologne at whole is determined with the unary-method from above.
</p>

In [ ]:
sf1_outer_boundaries = geop.GeoDataFrame(geometry=[sf_1_unary], crs=sf_1.crs)

# print(sf1_outer_boundaries.crs)
sf1_outer_boundaries

In [ ]:
cologne_center = sf1_outer_boundaries.geometry.centroid
cologne_center

In [ ]:
fig, ax = plt.subplots(1,1)


sf1_outer_boundaries.plot(edgecolor='black', color='gold', ax=ax)
cologne_center.plot(ax = ax, color='r', edgecolor='black', linewidth=0.4)

plt.title('Center of Cologne at a whole', fontsize=12)
plt.show()

### GeoSeries.distance()

<p>
allows to calculate the distance between two points.<br>
The "from" point is here the Cologne Cathedral (Kölner Dom), Cologne, Germany.<br>
Latitude and longitude coordinates are: 50.941357, 6.958307.<br>
The "other" points are the centroids from above.
</p> 

In [ ]:
cath_geo = geop.GeoSeries(data=Point((6.958307, 50.941357)),  crs="EPSG:4326")
print(cath_geo)
print(cath_geo.crs)

In [ ]:
cath_geo=cath_geo.to_crs(epsg=3857)
print(cath_geo.crs)
print(cath_geo)
print(cath_geo.crs)

Converting the the center point of Ensen into a GeoSeries.

In [ ]:
ensen_gs = geop.GeoSeries(data=ensen_c,  crs="EPSG:3857")

print(type(ensen_gs))
print(ensen_gs)
print(ensen_gs.crs)

Converting the the center point of Ensen into a GeoSeries.

In [ ]:
grengel_gs = geop.GeoSeries(data=[grengel_c],  crs="EPSG:3857")
print(type(grengel_gs))
print(grengel_gs)
print(grengel_gs.crs)

In [ ]:
dis_ensen = cath_geo.distance(other=ensen_gs)

dis_ensen_m = round(float(dis_ensen ),0)
print(dis_ensen_m)

dis_ensen_km = dis_ensen_m / 1000
print(dis_ensen_km)

According to Google maps the distance is around 8,5 km.<br>
The centroid is not identical with the maps distance.<br>
At least the "cath_geo.distance" is comprehensible and possible.<br>
What is the exact location of the centroid?<br>
(For now it is enough.)

In [ ]:
dis_greng = cath_geo.distance(other=grengel_gs)

dis_greng_m = round(float(dis_greng ),0)
print(dis_greng_m, 'm' )

dis_greng_km = dis_greng_m / 1000
print(dis_greng_km, 'km')

According to Google maps the distance is around 15 km.<br>
The centroid is not identical with the maps distance.<br>
At least the "cath_geo.distance" is comprehensible and possible.<br>
What is the exact location of the centroid?<br>
(For now it is enough.)

Distances between cathedral and every centroid.

In [ ]:
# sf_566_m.crs

In [ ]:
sf1_centroids = sf_566_m.geometry.centroid

print(type(sf1_centroids))
print(sf1_centroids.crs)
print(sf1_centroids.head(1))


In [ ]:
sf_1['centroid']  = sf1_centroids 

sf1_centroids_df = sf_1[['STT_NAME', 'centroid']]
print(type(sf1_centroids_df))
print(sf1_centroids_df.head(2))

In [ ]:
sf2_df_centers = \
geop.GeoDataFrame(sf1_centroids_df, crs='EPSG:3857', geometry='centroid')

sf2_df_centers.rename(columns={'STT_NAME':'district'}, inplace=True)
sf2_df_centers.to_crs(3857)


print(type(sf2_df_centers))
print(sf2_df_centers.shape)
print(sf2_df_centers.head(1))
print(sf2_df_centers.crs)

In [ ]:
distances_cathedral = []

for n, row in enumerate(sf2_df_centers.iterrows()):
    # print(type(row[1]))
    # print(row[1][1])
    distance = round(float(cath_geo.distance(other=row[1][1])),2)
    distances_cathedral.append(distance)
    
distances_cathedral[:2]

In [ ]:
sf2_df_centers['distance_m'] = distances_cathedral
sf2_df_centers['distance_km'] = [round(float(x/1000),2) for x in distances_cathedral]

sf2_df_centers.head(2)

In [ ]:
print(type(sf2_df_centers))
# print(sf2_df_centers.crs)
print(sf2_df_centers.shape)

#### Spatial joins

<p>
A spatial join uses binary predicates such as intersects and crosses to combine two GeoDataFrames based on the spatial relationship between their geometries.
</p> 

<p>
A common use case might be a spatial join between a point layer and a polygon layer where you want to retain the point geometries and grab the attributes of the intersecting polygons.<br>
<a href="https://geopandas.org/gallery/spatial_joins.html" target="_blank">geopandas.org</a> 
</p> 

<p>
The matching is achieved with the geometry columns, every GeoPandasDataframe has got.<br>
Without a geometry col a data structure cannot be a GeoPandasDataframe.<br>
The modi of joins in GeoPandas follows SQL logic.
</p>



In [ ]:
print(type(sf_2))
print(type(pm_geopd ))

print(sf_2.shape)
print(pm_geopd.shape)

print(sf_2.crs)
print(pm_geopd.crs)

In [ ]:
pm_intersect_dist = geop.sjoin(pm_geopd, sf_2, op='intersects')
print(type(pm_intersect_dist))
print(pm_intersect_dist.shape)

In [ ]:
pm_intersect_dist.head(1)

Top ten number of PM per district.

In [ ]:
pm_intersect_dist.STT_NAME.value_counts()[:10]

Plotting pm on districts.

<p>
This needs some preparation. Not all districts are included in the pm data set.
</p> 

In [ ]:
# sf_2.columns
# sf_2.index
# pm_intersect_dist.columns.to_list()

# index right from intersect
inter_indexr = pm_intersect_dist['index_right'].unique().tolist()
print(len(inter_indexr))

# subsetting sf_2 with the index from intersect
sf2_index = sf_2.iloc[sf_2.index[inter_indexr],:]
print(type(sf2_index))
print(sf2_index.shape)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,5))

sf2_index.plot(ax=ax) # plots districts with pm
pm_intersect_dist.plot(color='gold', edgecolor='black', ax=ax) # plot pm


plt.show()

<b>Selecting one record: Deutz</b><br>
<p>
Subsetting the complete record returns a pandas.core.series.Series.<br>
Subsetting the geometry returns geopandas.geoseries.GeoSeries'
</p> 


In [ ]:
deutz = sf2_index.loc[40,:]
deutz_geometry = geop.GeoSeries(sf2_index.loc[40,'geometry'])

print(type(sf2_index))
print(type(deutz))
print(type(deutz_geometry))

In [ ]:
print(deutz.index)
deutz 

Turning a pandas.core.series.Series to geopandas.geodataframe.GeoDataFrame.

In [ ]:
deutz_pdf=deutz.to_frame().transpose() 
print(type(deutz_pdf))
deutz_pdf.index
deutz_pdf

In [ ]:
deutz_gs = geop.GeoDataFrame(deutz_pdf, geometry='geometry')

In [ ]:
deutz_gs.plot(edgecolor='black', figsize=(12,3), color='fuchsia', aspect=0.5)
plt.show()

There are two district values: Deutz and Deutz I.<br>
Parking machines in these districts are subsetted below.

In [ ]:
deutz_boolean_mask = \
(pm_intersect_dist['Bezirk/Gebiet'] == 'Deutz' ) | (pm_intersect_dist['Bezirk/Gebiet'] =='Deutz I')

In [ ]:
pm_intersect_dist[deutz_boolean_mask].\
plot(edgecolor='black', figsize=(9,3), color='teal', aspect=0.5)
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,5))

deutz_gs.plot(edgecolor='black', figsize=(12,3), color='fuchsia', aspect=0.5, ax=ax)
pm_intersect_dist[deutz_boolean_mask].\
plot(edgecolor='black', figsize=(9,3), color='teal', aspect=0.5, ax=ax)

plt.title('Parking machines in the district of Deutz')
plt.show()

The points outside could belong to the district of Deutz I ?

### Combining Cologne shape polygons & scatter location plots: parking meters

<p>
into layered plots.
</p>

In [ ]:
colo_df_2.info()

In [ ]:
colo_df_2.Longitudes.describe() 

In [ ]:
fig, ax1 = plt.subplots(1,1, figsize=(10,8))
plt.tight_layout()

sf_2.plot(color='gold', edgecolor='black', linewidth=1, ax=ax1)

sns.scatterplot(x='Longitudes', y='Latitudes', 
                data=colo_df_2, color='black', edgecolor='red', ax=ax1)

ax1.set_title("Cologne parking meters")

plt.subplots_adjust(hspace=0.3, wspace=0.2)
plt.show()


In [ ]:
fig, ax1 = plt.subplots(1,1, figsize=(10,8))
plt.tight_layout()


sf_2.plot(edgecolor='black', linewidth=1, ax=ax1, column='STT_NAME', legend=True,
          legend_kwds={ 'bbox_to_anchor':(1, 1), 'ncol':3, 'loc':'upper left'})

sns.scatterplot(x='Longitudes', y='Latitudes', 
                data=colo_df_2, edgecolor='black', ax=ax1)
ax1.set_title("Cologne parking meters colored by district")

plt.subplots_adjust(hspace=0.3, wspace=0.2)

plt.show()


# Creating interactive maps of Cologne with Folium

<p>
, which is Python package based on the leaflet.js library<br>
or in words of the <a href="https://python-visualization.github.io/folium/" target="_blank">Folium 0.12.1 documentation:</a>
</p> 

<p>
"Folium makes it easy to visualize data that’s been manipulated in Python on an interactive leaflet map. It enables both the binding of data to a map for choropleth visualizations as well as passing rich vector/raster/HTML visualizations as markers on the map."
</p>

<p>
Below a map is created centered around the Cologne cathedral.<br>
The cathedral is located in the district of Altstadt-Nord.
</p> 

In [ ]:
cathedral_location = [50.941357, 6.958307]

# Zoom centers the map around the location
cathedral = folium.Map(location=cathedral_location, zoom_start=18)

In [ ]:
display(cathedral)

#### Customizing folium maps
<p>
Resizing folium maps.
</p> 


In [ ]:
f = folium.Figure(width=400, height=300)

cathedral_map_resized = folium.Map(location=cathedral_location, zoom_start=14,
                                   tiles='openstreetmap').add_to(f)
display(cathedral_map_resized )

#### Add (district) shape borders to the Folium map.

<p>
At first the shape is selected from the data set.
</p> 

In [ ]:
altstadt_nord = sf_2.loc[sf_2.STT_NAME=='Altstadt/Nord']

print(type(altstadt_nord))
altstadt_nord

Getting the polygon coordinates.<br>
Not necessary here.

In [ ]:
altn_poly = np.array(altstadt_nord.geometry)[0]
print(type(altn_poly))
ext = altn_poly.exterior.coords[:-1]
print(type(ext))
altn_poly.exterior.coords

In [ ]:
altstadt_nord.plot(edgecolor='black', color='orange')
plt.title('Altstadt Nord')
plt.show()

In [ ]:
f = folium.Figure(width=600, height=300)

cathedral_map_3 = folium.Map(location=cathedral_location, zoom_start=12.5,
                                   tiles='openstreetmap').add_to(f)

In [ ]:
altstadt_nord_json = altstadt_nord.to_json()
altstadt_nord_json[:100]

Folium maps is based on Leaflet.js.<br>
Both are using GeoJSON.<br>
Folium maps are customizable by using GeoJSON-frames.<br>
<a href="https://leafletjs.com/examples/geojson/" target="_blank">leafletjs</a> 


In [ ]:
alt_shape =    folium.GeoJson(data=altstadt_nord_json, 
               style_function= \
               lambda x: {'fillColor': 'orange',
                          "color": "#ff7800",
                          "weight": 5,
                          "opacity": 0.65,
                          "stroke":True})

In [ ]:
str(altstadt_nord['STT_NAME'].values)

#### Add a popup to the folium map

In [ ]:
cathedral_description = \
"""
<strong>Cologne Cathedral</strong> (German: Koelner Dom, officially Hohe Domkirche Sankt Petrus, 
English: Cathedral Church of Saint Peter) is a Catholic cathedral in Cologne, 
North Rhine-Westphalia, Germany. It is the seat of the Archbishop of Cologne 
and of the administration of the Archdiocese of Cologne. 
It is a renowned monument of German Catholicism and Gothic architecture 
and was declared a World Heritage Site in 1996.
<a href="https://en.wikipedia.org/wiki/Cologne_Cathedral" target="_blank">Wikipedia</a> 
"""


folium.Popup(cathedral_description).add_to(alt_shape)

In [ ]:
alt_shape.add_to(cathedral_map_3)

In [ ]:
display(cathedral_map_3)

#### Folium map of Cologne

<p>
centered around Cologne centroid.
</p>

<strong>Removing diacritical marks using Python</strong>

<p>
In this case the German vowel mutations are removed.<br>
<a href="https://stackoverflow.com/questions/48445459/removing-diacritical-marks-using-python" 
target="_blank">Stack</a> 
</p>



In [ ]:
# sf_1.crs
sf_356 = sf_1.copy()

sf_356['centroid'] = sf_1.geometry.to_crs(3857).centroid
sf_356['centroid'] = sf_356['centroid'].to_crs(4326)
# print(sf_356.centroid.crs)

sf_356.head(1)

In [ ]:
# sf_356.STT_NAME.unique
STT_NAME_idx = sf_356.columns.get_loc('STT_NAME')

In [ ]:
unvoweled_names = []

for i in sf_356.iterrows():
    
    print(i[1][STT_NAME_idx])
    unvoweled_name = unidecode.unidecode(i[1][STT_NAME_idx])
    unvoweled_names.append(unvoweled_name)
    
unvoweled_names[:5]

In [ ]:
sf_356['STT_NAME'] = unvoweled_names

It does not exactly what I wanted.<br>
I wanted to replace vowels with ae and so on.

In [ ]:
cologne_center_4326 = cologne_center.to_crs(4326)
cologne_center_geocoord =[float(cologne_center_4326.y), 
                          float(cologne_center_4326.x)]
cologne_center_geocoord

<strong>
Add title to Folium map.
</strong>

In [ ]:
title_colog_map_districts = """Cologne with districts."""

title_colog_map_districts = \
'''
<h2 align="center" style="font-size:14px" font-style="italic"><b>{}</b></h2>
           
'''.format(title_colog_map_districts)

<strong>
Add pop up with Cologne info.
</strong>

<p>
Special signs in the text of the popup must be escaped. Otherwise an error is returned.
.<br>

In [ ]:
colog_descript = \
"""<strong>Cologne</strong> is the largest city of Germanys most populous state of North 
Rhine-Westphalia and the fourth-most populous city in Germany. 
With 3.6 million people in the urban region and 1.1 million inhabitants within its city proper,
Cologne is the largest city on the river Rhine and also the most populous city of both 
the Rhine-Ruhr Metropolitan Region and the Rhineland.
<a href="https://en.wikipedia.org/wiki/Cologne"
target="_blank"">Wikipedia</a>"""

Again solving conflicts concerning crs.

In [ ]:
sf_1.head(1)

In [ ]:
geo_index = sf_356.columns.get_loc('centroid')
geo_index_title = sf_356.columns.get_loc('STT_NAME')

Extracting the geo coordinates of the district centroids.

In [ ]:
geocoords_colog = []

for row in sf_356.iterrows():
    row_values = row[1]
    geocoord_colog  = [row_values[geo_index].y, row_values[geo_index].x]
    geocoords_colog.append(geocoord_colog)
    
geocoords_colog[:3]

Creating the map with markers and popups.

In [ ]:
f_colog = folium.Figure(width=600, height=300)


colog_map = folium.Map(location = 
                       cologne_center_geocoord, 
                       zoom_start=14, tiles='openstreetmap')\
                       .add_to(f_colog)


colog_borders = folium.GeoJson(sf_1.geometry, style_function= \
                              lambda x: {'fillColor': 'limegreen',
                              "color": "black",
                              "weight": 1,
                              "opacity": 0.8,
                              "stroke":True}).add_to(colog_map)

# This creates the markers for every district
# It is important to place the marker and pop up inside the for loop

for row in sf_356.iterrows():
    row_values = row[1]
    geocoord_colog  = [row_values[geo_index].y, row_values[geo_index].x]
    
# This creates the pop up "name" for every district

    popup = row_values[geo_index_title]
    
    colog_districts_marker = folium.Marker(location = geocoord_colog, popup = popup)
    colog_districts_marker.add_to(colog_map)

colog_map.get_root().html.add_child(folium.Element(title_colog_map_districts))
folium.Popup(colog_descript).add_to(colog_borders)



In [ ]:
display(colog_map)

#### Folium map of Cologne wit parking meters

Remove vowels.

In [ ]:
def diacritical(row, v):
        return unidecode.unidecode(row[v])

Surely there are better methods with replacement out there.

In [ ]:
# colo_df_2.columns
# colo_df_2_34534 = colo_df_2.copy()
# colo_df_2_34534.columns

In [ ]:
colo_df_2['Aufstellort'] = \
colo_df_2.apply(diacritical, args=(['Aufstellort']), axis=1)

In [ ]:
# colo_df_2['Aufstellort']
# colo_df_2.head(1)
# colo_df_2.info()

Check geocoords of Cologne center.

In [ ]:
cologne_center_geocoord
colo_df_2.head(1)

This is the for loop for adding geo data to the map.

In [ ]:
geocoords_pm = []
locations_pm = []

for row in colo_df_2.iterrows():
    row_value = row[1]
    geocoord_pm = [row_value['Latitudes'], row_value['Longitudes']]
    location_pm = row_value['Aufstellort']
    geocoords_pm.append(geocoord_pm)
    locations_pm.append(location_pm )
    
print(len(geocoords_pm))    
geocoords_pm[:2]

print(len(locations_pm))
locations_pm[:4]

In [ ]:
f_parking = folium.Figure(width=700, height=350)

# Create a map centered around center of cologne
# The geocoordinates need the form list(latitude, longitude)
map_pm = folium.Map(location=cologne_center_geocoord, 
                    zoom_start=16, tiles='openstreetmap').add_to(f_parking)

map_pm_district_borders = folium.GeoJson(sf_1.geometry,
                                         style_function = \
                                         lambda x: {'color': 'black', 'weight': 0.5, 
                                                    'stroke': True}).add_to(map_pm)

for row in colo_df_2.iterrows():
    row_value = row[1]
    
    # This is for the parking meters markers
    geocoord_parkingmeters = [row_value['Latitudes'], row_value['Longitudes']]
    # This adds the popup to the marker with the name of the location
    pm_popup = row_value['Aufstellort']
    
    
    folium.Marker(location=geocoord_parkingmeters, popup=pm_popup,
                  icon = folium.Icon(color='green', 
                                     icon_color='white', icon='anchor', 
                                     prefix='fa')).add_to(map_pm)

In [ ]:
display(map_pm)

## Quick reproduction: hospitals in Cologne

Import data with request API for http calls.

In [ ]:
hospitals_endpoint_url = \
'https://geoportal.stadt-koeln.de/arcgis/rest/services/Stadtplanthemen/MapServer/4/query?text=&geometry=&geometryType=esriGeometryPoint&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&objectIds=&where=objectid+is+not+null&time=&returnCountOnly=false&returnIdsOnly=false&returnGeometry=true&maxAllowableOffset=&outSR=4326&outFields=*&f=json'

hospitals_rawdata = requests.get(hospitals_endpoint_url).json()

In [ ]:
print(type(hospitals_rawdata))

Flatten the nested dict and thereby remove attribute meta data.

In [ ]:
# print(hospitals_rawdata)
print(hospitals_rawdata.keys())
print(type(hospitals_rawdata['fields']))
print(hospitals_rawdata['fields'][:2])
print(type(hospitals_rawdata['features']))
print(hospitals_rawdata['features'][0])

In [ ]:
pm_455 = pd.json_normalize(pm_data, record_path=['result', 'records'])

In [ ]:
hospitals_df = pd.json_normalize(hospitals_rawdata, record_path=['features'])

print(type(hospitals_df))
print(hospitals_df.info())
print(hospitals_df.head())

Cleaning col names.

In [ ]:
hospitals_df.rename(columns={'geometry.x':'Longitude', 'geometry.y':'Latitude'}, inplace=True)

In [ ]:
hospitals_df.columns

Constructing a geo df.

In [ ]:
hospitals_df_2 = hospitals_df.copy()

In [ ]:
hospitals_df_2['geometry'] = \
hospitals_df_2.apply(lambda y: Point((y.Longitude, y.Latitude)), axis=1)

print(type(hospitals_df_2))
print(hospitals_df_2.loc[:2, 'geometry'])

In [ ]:
hospitals_geo = geop.GeoDataFrame(hospitals_df_2, crs="EPSG:4326" , geometry='geometry')
hospitals_geo.columns = hospitals_geo.columns.str.replace('attributes.', '', regex=False)

print(type(hospitals_geo))
print(hospitals_geo.crs)
print(hospitals_geo.columns.tolist())

In [ ]:
hospitals_geo.head(1)

Plot 

In [ ]:
fig, (ax1, ax2) = plt.subplots(2,1, figsize=(10,15))
plt.tight_layout()

sf_2.plot(ax=ax1)
hospitals_geo.plot(ax=ax1, column='NAME', aspect=0.5, edgecolor='black', legend=True,
                   legend_kwds={'ncol':5, 'bbox_to_anchor':(0.5, -0.1), 'loc':'upper center'})
ax1.set_title('Hospitals locations in Cologne')


sf_2.plot(ax=ax2, column='STT_NAME', aspect=1, legend=True,
         legend_kwds={'ncol':6, 'bbox_to_anchor':(0.5, -0.05), 'loc':'upper center'})
hospitals_geo.plot(ax=ax2, column='NAME', aspect=0.7, edgecolor='black', legend=False)
ax2.set_title('Hospitals locations in Cologne by district')


plt.subplots_adjust(hspace=0.05, wspace=0.5)
plt.show()


#### Spatial joins

In [ ]:
print(hospitals_geo.crs)
print(sf_2.crs)

In [ ]:
hospitals_intersect_dist = geop.sjoin(hospitals_geo, sf_2, op='intersects')
print(type(hospitals_intersect_dist))
print(hospitals_intersect_dist.shape)
print(hospitals_intersect_dist.columns.tolist())
print(hospitals_intersect_dist.STT_NAME.unique())

inter_indexr_hospitals = hospitals_intersect_dist['index_right'].unique().tolist()
print(len(inter_indexr_hospitals))
sf2_index_hospitals = sf_2.iloc[sf_2.index[inter_indexr_hospitals],:]
print(type(sf2_index_hospitals ))
print(sf2_index_hospitals.shape)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,5))

sf2_index_hospitals.plot(ax=ax) # plots districts with pm
hospitals_intersect_dist.plot(column='NAME', 
                              edgecolor='black', ax=ax,
                              legend=True,
                              legend_kwds={'ncol':3, 'bbox_to_anchor':(0.5, -0.1), 'loc':'upper center'}) 


plt.show()

Deutz

In [ ]:
deutz_Hospitals_boolean_mask = \
(hospitals_intersect_dist['STT_NAME'] == 'Deutz' )

hospital_deutz = hospitals_intersect_dist[deutz_Hospitals_boolean_mask]
hospital_deutz

In [ ]:
deutz_hospitals_locations = \
hospital_deutz.plot(edgecolor='black', figsize=(8,4), color='teal', aspect=0.2)

deutz_hospitals_locations
plt.show()

In [ ]:
hospital_deutz

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,5))

deutz_gs.plot(edgecolor='black', color='fuchsia', aspect=0.5, ax=ax)
hospital_deutz.plot(edgecolor='black', ax=ax, column='NAME', legend=True,
                    legend_kwds={'ncol':1, 'bbox_to_anchor':(0.5, -0.1), 'loc':'upper center'})

plt.title('Hospitals in the district of Deutz')
plt.show()

Only one hospital in Deutz.

Altstadt/Nord

In [ ]:
altnord_Hospitals_boolean_mask = \
(hospitals_intersect_dist['STT_NAME'] == 'Altstadt/Nord' )

hospital_altnord = hospitals_intersect_dist[altnord_Hospitals_boolean_mask]
hospital_altnord

In [ ]:
altnord_shape= sf2_index_hospitals[sf2_index_hospitals.STT_NAME=='Altstadt/Nord']
print(type(altnord_shape))
altnord_shape.plot(edgecolor='black', color='yellowgreen')
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,5))

altnord_shape.plot(edgecolor='black', linewidth=2, color='yellowgreen', ax=ax)
hospital_altnord.plot(column='NAME', edgecolor='black', ax=ax, legend=True,
                      legend_kwds={'ncol':1, 'bbox_to_anchor':(0.5, -0.1), 'loc':'upper center'})

plt.title('Hospitals in the district of Altstadt/Nord', fontsize=12)
plt.show()

### Calculating geospatial attributes.

Geo centroids

In [ ]:
# hospitals_geo.head(1)
print(hospitals_geo.crs)

hospitals_geo_2 = hospitals_geo.to_crs(3857)
hosp_centroids = hospitals_geo_2.geometry.centroid

print(hosp_centroids.crs)

In [ ]:
hosp_centroids[:2]

#### Geo distances

<p>
from cathedral.
</p>

In [ ]:
def distances(row, p=cath_geo):
    pr = p.distance(other=row)
    return round(pr, 2)

In [ ]:
hospitals_geo['Distances_cathedral m']=hosp_centroids.apply(distances)

In [ ]:
hospitals_distances = hospitals_geo[['NAME', 'Distances_cathedral m']]
hospitals_distances = hospitals_distances.copy()
hospitals_distances['km'] = round(hospitals_distances['Distances_cathedral m'] / 1000, 2)
hospitals_distances.sort_values(by='km', ascending=False).head()

From center of Cologne.

In [ ]:
hospitals_geo_2323 = hospitals_geo.copy()

In [ ]:
# cath_geo
# cath_geo.crs
# cologne_center.crs
# hosp_centroids

In [ ]:
cologne_center_3857 = cologne_center.to_crs(3857)
cologne_center_3857

In [ ]:
hospitals_geo_2323['Distances_center_m']= \
hosp_centroids.apply(distances, args=([cologne_center_3857]))

In [ ]:
hospitals_geo_2323.head(1)

#### Folium map of Lindental district around university clinics

In [ ]:
# hospitals_geo.NAME.unique()
uniklinik = hospitals_geo[hospitals_geo.NAME == 'Universitätskliniken']
# uniklinik.columns
uniklinik

In [ ]:
uni_geocoords = [float(uniklinik.geometry.y), float(uniklinik.geometry.x)]
print(uni_geocoords)
print(uniklinik.crs)

f = folium.Figure(width=550, height=250)

uniklinik_map = folium.Map(location=uni_geocoords, zoom_start=15,
                           tiles='openstreetmap').add_to(f)


folium.Marker(location=uni_geocoords, popup='Universitaetskliniken:<br>Joseph-Stelzmann-Str. 9',
              icon = folium.Icon(icon='hospital-symbol', prefix='fa',
                                 color='green')).add_to(uniklinik_map)

In [ ]:
display(uniklinik_map)

Displaying borders of Lindenthal.

In [ ]:
# hospitals_intersect_dist[hospitals_intersect_dist.NAME == 'Universitätskliniken']
Lindenthal = sf_2.loc[sf_2.STT_NAME=='Lindenthal']

print(type(Lindenthal))
Lindenthal

In [ ]:
Lindenthal_json = Lindenthal.to_json()

Lindenthal_borders  =   folium.GeoJson(data=Lindenthal_json, 
                        style_function= \
                        lambda x: {'fillColor': 'crimson',
                          "color": "darkblue",
                          "weight": 7,
                          "opacity": 0.6,
                          "stroke":True})

In [ ]:
folium.Popup(""" Lindenthal is a city district of the City of Cologne in Germany. It has about 153,000 inhabitants (as of December 2019) 
and covers an area of 41.8 square km. Many parts of Lindenthal are dominated by academic 
and research campuses, primarily linked to the University of Cologne and the German
Sport University.""" ).add_to(Lindenthal_borders)

f = folium.Figure(width=550, height=250)

uniklinik_map_2 = folium.Map(location=uni_geocoords, 
                             zoom_start=12,
                             tiles='openstreetmap').add_to(f)


Lindenthal_borders.add_to(uniklinik_map_2)

display(uniklinik_map_2)

#### Folium map of hospitals in Cologne

In [ ]:
hospitals_geo.head(1)
hospitals_geo.shape[0]

In [ ]:
hospitals_geo['NAME'] = \
hospitals_geo.apply(diacritical, args=(['NAME']), axis=1)

This is another approach to iterate over the geo coords.

In [ ]:
geocoords_hospitals = []
hospital_names = []

for i in range(0, hospitals_geo.shape[0]):
    geocoords_hospital = [hospitals_geo.iloc[i]['Latitude'], hospitals_geo.iloc[i]['Longitude']]
    geocoords_hospitals.append(geocoords_hospital)
    
    hospital_name = hospitals_geo.iloc[i]['NAME']
    hospital_names.append(hospital_name)

In [ ]:
print(geocoords_hospitals[:2])
print(hospital_names[:4])

In [ ]:
f_hospitals = folium.Figure(width=500, height=300)

hospitals_title= """Hospitals in Cologne"""

hospitals_title_2 = \
'''<h1 align="center" style="font-size:16px" font-style="italic"><b>{}</b></h2>
'''.format(hospitals_title)


map_hospitals = folium.Map(location=cologne_center_geocoord,
                           width='100%', height='100%', left='0%', top='0%', 
                           position='relative', tiles='OpenStreetMap', 
                           attr=None, min_zoom=2, max_zoom=18, 
                           zoom_start=11, min_lat=- 90, max_lat=90, min_lon=- 180, 
                           max_lon=180, max_bounds=False, crs='EPSG3857', 
                           control_scale=False, prefer_canvas=False, 
                           no_touch=False, disable_3d=False, png_enabled=False)

map_hospitals_districts = folium.GeoJson(sf_1.geometry,
                                         style_function = \
                                         lambda x: {'color': 'black', 'weight': 0.5, 
                                                    'stroke': True}).add_to(map_hospitals)

for i in range(0, hospitals_geo.shape[0]):
    geocoords_hospital = [hospitals_geo.iloc[i]['Latitude'], hospitals_geo.iloc[i]['Longitude']]
    hospital_name = hospitals_geo.iloc[i]['NAME']
    
    folium.Marker(location=geocoords_hospital, popup=hospital_name,
                  icon = folium.Icon(color='green', icon_color='white', icon='heart', 
                                     angle=0, prefix='glyphicon')).add_to(map_hospitals)
 

 
map_hospitals.get_root().html.add_child(folium.Element(hospitals_title_2))

In [ ]:
display(map_hospitals)

## Choropleth map
<p>
"(from Greek χῶρος choros 'area/region' and πλῆθος plethos 'multitude') is a type of thematic map in which a set of pre-defined areas is colored or patterned in proportion to a statistical variable that represents an aggregate summary of a geographic characteristic within each area, such as population density or per-capita income." (Wikipedia)
</p> 

<p>
The definition above makes clear, a statistical variable is necessary.<br>
Statistical variables are derived from given information for example by calculating a ratio sythentically, based on new introduced variables from other data sets, or both.
</p> 

<p>
The data set including the statistical variable here is the inhabitant development of Cologne by districts.
</p> 

### Calculating statistical variables 
for expressing them in the Choropleth

In [ ]:
# Einwohnerentwicklung_Stadtteil_6.csv
inhabitants =  pd.read_csv('datasets/Einwohnerentwicklung_Stadtteil_6.csv',
                           delimiter=',', encoding='utf-8')

In [ ]:
print(type(inhabitants))
print(inhabitants.shape)
print(inhabitants.info())

In [ ]:
inhabitants.head(5)

In [ ]:
# Vowels are displayed correctly
inhabitant_districts = inhabitants.Stadtteil.unique()
inhabitant_districts = np.sort(inhabitant_districts)

print(type(inhabitant_districts))
print(inhabitant_districts[:5])
print(len(inhabitant_districts))

Are the districts matching within data sets?

In [ ]:
sf_1.head(1)

In [ ]:
sf_1_colog_districts = sf_1.STT_NAME.unique()
sf_1_colog_districts  = np.sort(sf_1_colog_districts)

print(sf_1_colog_districts[:3])
print(len(sf_1_colog_districts))
print(type(sf_1_colog_districts))

In [ ]:
compare_districts = \
pd.DataFrame(zip(inhabitant_districts, sf_1_colog_districts ), columns=['Inhabitants', 'sf_1'])


In [ ]:
compare_districts.head()

In [ ]:
compare_districts['Match_?'] = \
compare_districts.apply(lambda x: 1 if x.Inhabitants == x.sf_1 else 0, axis=1 )

In [ ]:
compare_districts.head(2)

Number of matches?

In [ ]:
compare_districts['Match_?'].value_counts()

In [ ]:
mismatches = compare_districts[compare_districts['Match_?'] == 0]
mismatches

The mismatches are caused by hyphens an forward slash.<br>
Below the hyphens are replaced with forward slashes.

In [ ]:
mis_list = mismatches.Inhabitants.to_list()
mis_list 

# print(inhabitants.Stadtteil)

In [ ]:
matching_df = \
inhabitants.apply(lambda x: 1 if x.Stadtteil in  mis_list else 0, axis=1)
matching_df[:5]

In [ ]:
mis_dropindex = mismatches.sf_1.reset_index(drop=True)

In [ ]:
inhabitants.loc[matching_df==1, 'Stadtteil'] = mis_dropindex
inhabitants.head(4)

Time series plots.

In [ ]:
# index represents the groupby
inhab_mline = inhabitants.set_index('Stadtteil').drop('Nr.', axis=1)
inhab_mline.head(1)

In [ ]:
inhab_mline_T = inhab_mline.T
inhab_mline_T.head(1)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15,8))
plt.tight_layout()

inhab_mline_T.plot(legend=True, ax=ax, 
                 title="Inhabitant development over time by district")

# bbox_to_anchor= (-0.03 = right to left, -0.1 = upper to lower) - adjustment of legend
ax.legend(bbox_to_anchor= (-0.07, -0.1), loc="upper left", ncol=8)
ax.set_xlabel('Year', fontsize=14)
ax.set_ylabel('Inhabitats (n)', fontsize=14)


plt.show()

In [ ]:
color_list=['#004200', '#0000bd', '#46f0f0', '#f032e6', '#bcf60c', '#000080',
            '#f58231', '#ffbb3d', '#0000FF', '#242400', '#bcf60c', '#fabebe',
            '#000042', '#6e00c2', '#616161', '#0f0f0f']

color_list_2 = np.repeat(color_list, 10)

Multiple line plot - one plot over time for every district.

In [ ]:
list_keys = inhab_mline.index.to_list() 

fig = plt.figure(figsize=(100,700))
fig.subplots_adjust(wspace=0.1, hspace=0.6, bottom=0, top=0.5)


for u,(i,j) in enumerate(zip(range(1,87), list_keys)):
        ax = fig.add_subplot(40, 3, i)
        # inhab_mline.loc['Altstadt/Nord', :]
        inhab_mline.loc[j, :].plot(legend=False, ax=ax, color=color_list_2[i],
                                   title=list_keys[u])
   

###  Average inhabitant number from 2000 to 2012 per district: metric - mean inhabitants

In [ ]:
# print(inhabitants.head())
years= inhabitants.iloc[:, [2,3,4,5]]
print(years.info())

In [ ]:
means_years = []

for row in inhabitants.iterrows():
    # print(row[1])
    mean_year = round((row[1][2] + row[1][3] + row[1][4] + row[1][5]) / 4,0)
    # print(type(row[1][2]))
    means_years.append(mean_year)

print(len(means_years))
print(means_years[:5])

In [ ]:
inhabitants_2 = inhabitants.copy()

In [ ]:
inhabitants_2.loc[:,'Mean_Inhabitants'] = means_years

In [ ]:
inhabitants_2.head()

In [ ]:
inhab_merge = inhabitants_2.iloc[:, [0,1,6]]
inhab_merge.head(1)

Sum inhabitants of Cologne by year.

In [ ]:
# cols to index - years to index
years_index = inhabitants.drop('Nr.', axis=1).T

In [ ]:
# Stadtteile as col names
years_index = \
years_index.rename(columns = 
                   {col:name for col,name in zip(years_index.columns, inhabitants.Stadtteil)})

In [ ]:
years_index.drop('Stadtteil', inplace=True)

In [ ]:
years_index.head()

In [ ]:
years_index.iloc[0,:].sum()

Calculating row sums.

In [ ]:
np.sum(years_index.iloc[0,:])

In [ ]:
def rowsums(row):
    return np.sum(row)

<strong>Apply function to each row.</strong>
 
<p>
axis{0 or ‘index’, 1 or ‘columns’}, default 0<br>
Axis along which the function is applied:<br>
0 or ‘index’: apply function to each column.<br>
1 or ‘columns’: apply function to each row.
</p> 

In [ ]:
years_index['Sum'] = years_index.apply(rowsums, axis=1)

In [ ]:
years_index.iloc[:,[0,5,-1]]

In [ ]:
sf_1.columns
sf_1.head(1)

In [ ]:
cologne_inhabitants = \
sf_1.merge(inhab_merge, left_on='STT_NAME', right_on='Stadtteil')

Adding the area as a another variable.

In [ ]:
# cologne_inhabitants.geometry.crs
conversion_m_km = 1000000
cologne_inhabitants.loc[:, 'Area_km'] = round(cologne_inhabitants.geometry.to_crs('EPSG:3857').area / conversion_m_km,2)

In [ ]:
# Check
inhabitants_2[inhabitants_2.Stadtteil == 'Flittard']

<strong>Calculating population density</strong>
<p>
A population density is defined as the total number of people per unit of area:<br>
D = P / A <br>
Where D is the density<br>
P is the population number<br>
A is the area.<br>
<a href="https://calculator.academy/population-density/" target="_blank">Population Density Calculator</a> 
</p> 



In [ ]:
cologne_inhabitants['population_density'] = \
round(cologne_inhabitants['Mean_Inhabitants'] / cologne_inhabitants['Area_km'],2)

In [ ]:
print(type(cologne_inhabitants))
cologne_inhabitants.head(1)

### Choropleth maps

In [ ]:
fig = plt.figure(figsize=(15,15))
fig.tight_layout()
#plt.subplots_adjust(wspace=0, hspace=0.6)

ax = fig.add_subplot(221)


cologne_inhabitants.plot(column='Area_km', legend=True, ax=ax, figsize=((8,4)),
                         cmap='coolwarm')
ax.set_title('Cologne districts colored \n by area size', fontsize=16)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

ax2 = fig.add_subplot(222)
cologne_inhabitants.plot(column='Mean_Inhabitants', legend=True, ax=ax2, figsize=((8,4)), 
                         cmap='Greens')
ax2.set_title('Cologne districts colored \n by mean inhabitants numbers', fontsize=16)
ax2.set_xlabel('Longitude')
ax2.set_ylabel('Latitude')

ax3 = fig.add_subplot(223)
cologne_inhabitants.plot(column='population_density', legend=True, ax=ax3, figsize=((8,4)), 
                         cmap='Oranges')
ax3.set_title('Cologne districts colored \n by population_density', fontsize=16)
ax3.set_xlabel('Longitude')
ax3.set_ylabel('Latitude')

plt.show()

Merging the parking meters data df with the cologne_inhabitants df.

Selecting the relevant variables.

In [ ]:
inhab_reduced = cologne_inhabitants[['Stadtteil', 'Mean_Inhabitants', 'Area_km', 
                                     'population_density']]
inhab_reduced.head(1)

Merging the geo df created with a spatial join from above with the inhabitant df.<br>
This confirms what a powerful tool spatial joins are.

In [ ]:
pm_inter_inhab = \
pm_intersect_dist.merge(inhab_reduced, left_on='STT_NAME', right_on='Stadtteil')

In [ ]:
sf1_geo_name = sf_1[['STT_NAME', 'geometry']]

In [ ]:
sf1_geo_name = sf1_geo_name.rename(columns = {'geometry':'district_geometry'})
sf1_geo_name.head(1)

In [ ]:
pm_inter_inhab_districts = pm_inter_inhab.merge(sf1_geo_name, on = 'STT_NAME')

In [ ]:
cologne_inhabitants.head(1)

In [ ]:
choro_pm = pm_inter_inhab_districts.copy()
choro_pm.rename(columns={'geometry':'geometry_pm', 'district_geometry': 'geometry'},
                inplace=True)

In [ ]:
# print(choro_pm.geometry.crs)
# print(choro_pm.geometry_pm.crs)
# print(cologne_inhabitants.head(1))
print(choro_pm.crs)
choro_pm.head(1)


In [ ]:
### Chorpleth plotting based on statistical variables.

In [ ]:
choro_pm.shape

In [ ]:
choro_pm.drop(['STT_NAME', 'Abschnitt/-von', 'Abschnitt/-bis'], 
              axis=1, inplace=True)

In [ ]:
print(choro_pm.shape)
print(type(choro_pm))

In [ ]:
choro_pm_gb_Gebiet = choro_pm.groupby('Bezirk/Gebiet')['Bezirk/Gebiet'].count()
choro_pm_gb_Gebiet_df = choro_pm_gb_Gebiet.to_frame(name='pm_per_Bezirk')
choro_pm_gb_Gebiet_df.reset_index(inplace=True)

# choro_pm_gb_Gebiet_df.index.rename('')
print(type(choro_pm_gb_Gebiet_df))
print(choro_pm_gb_Gebiet_df.shape)
choro_pm_gb_Gebiet_df.head(2)

In [ ]:
choro_pm_233 = \
choro_pm[['Bezirk/Gebiet', 'Stadtteil', 
          'Mean_Inhabitants', 'Area_km', 'population_density',
          'geometry']]

In [ ]:
print(type(choro_pm.geometry))

In [ ]:
choro_pm.geometry[:3]

In [ ]:
# choro_pm.columns

In [ ]:
choro_pm.groupby('Bezirk/Gebiet')['PSA-Nr.'].count()[:10]

In [ ]:
choro_pm.to_crs(3857).plot()
plt.show()

In [ ]:
# Like above - check - works
cologne_inhabitants.plot(column='Area_km', legend=True)
plt.title('Districts colored by area size', fontsize=12)
plt.show()

In [ ]:
choro_pm.to_crs(3857).plot(column='Area_km', legend=True)
plt.title('Districts with parking meters \n colored by area size', fontsize=12)
plt.show()

In [ ]:
# Remind the crs!!!
fig = plt.figure()
ax=fig.add_subplot(111)
choro_pm.geometry.to_crs(3857).plot(ax=ax, cmap='YlGn')
choro_pm.geometry_pm.to_crs(3857).plot(ax=ax, color='r', edgecolor='black')
ax.set_title('Districts with parking meters', fontsize=12)
plt.show()

In [ ]:
# Remind the crs!!!
fig = plt.figure()
ax=fig.add_subplot(111)
choro_pm.to_crs(3857).plot(column='population_density', ax=ax, cmap='PuOr', legend=True)
choro_pm.geometry_pm.to_crs(3857).plot(ax=ax, color='r', edgecolor='black')
ax.set_title('Districts with parking meters \n colored by population density', fontsize=12)
plt.show()

Is there a correlation between parking meters and population density?

In [ ]:
# Remind the crs!!!
fig = plt.figure()
ax=fig.add_subplot(111)
choro_pm.to_crs(3857).plot(column='Mean_Inhabitants', ax=ax, cmap='Spectral', legend=True)
choro_pm.geometry_pm.to_crs(3857).plot(ax=ax, color='r', edgecolor='black')
ax.set_title('Districts with parking meters \n colored by Mean_Inhabitants', 
             fontsize=12)
plt.show()

### Choropleth layers on Folium maps

<p>
Choropleth can be easily created by binding the data between Pandas DataFrames/Series and Geo/TopoJSON geometries. Color Brewer sequential color schemes are built-in to the library, and can be passed to quickly visualize different combinations.
</p> 
<a href="https://python-visualization.github.io/folium/quickstart.html#Choropleth-maps" target="_blank">Github</a> 

<p>
Two arguments of folium.choropleth are not self-explanatory.
</p> 

<ul>
  <li>
  columns - a list of columns: one that corresponds to the polygons and one that has the
  value to plot</li>
  <li>key_on - a GeoJSON variable to bind the data to (always starts with feature )</li>
</ul> 

<p>
First we check the data frame. A special focus like always, when working with geopandas must 
be the CRS.</p> 

In [ ]:
choro_pm = choro_pm.rename(columns={'Bezirk/Gebiet':'Bezirk'})
choro_pm.info()

In [ ]:
choro_pm.head(1)

<p>
The column geometry is the location of the parking meters.<br>
This is useful, when location are set with markers, but can raise conflicts.<br>
Geopandas is looking for geometry one geometry column sometimes.<br>
At this point the column is therefore dropped.
</p> 

In [ ]:
choro_pm = choro_pm.drop('geometry_pm', axis=1)

In [ ]:
choro_pm = choro_pm.drop(columns=['Tagesgebuehr'])
choro_pm = choro_pm.dropna()
choro_pm.isnull().sum()

In [ ]:
choro_pm.head(1)

In [ ]:
choro_pm.to_file("choro_pm_test.geojson", driver='GeoJSON')


In [ ]:
mode12 = choro_pm.mode
print(type(mode12))

In [ ]:
print(choro_pm.geometry.crs)
print(choro_pm.geometry[:3])

<p>
<b>Checking the GeoJson.</b><br>
If the Json does not work folium.choropleth will not work.<br>
The geo_data argument does not work without a GeoJson.
</p> 

In [ ]:
choro_complete_json =  choro_pm.to_json()

In [ ]:
choro_complete_json[:1000]

In [ ]:
choro_pm_dict = choro_pm.to_dict()
choro_pm_dict.keys()

In [ ]:
choro_pm.shape


In [ ]:
choro_pm.shape[0]

In [ ]:
choro_pm.head(2)

In [ ]:
print(type(choro_pm.geometry[1]))

#### Choropleth map of mean inhabitants by district

In [ ]:
map_pm_3 = folium.Map(location=cologne_center_geocoord, zoom_start=10)
# folium.TileLayer('CartoDB positron', name="Light Map",control=False).add_to(map_pm_3)

map_pm_3.choropleth(
geo_data=choro_pm,
name='Choropleth',
data=choro_pm,
columns=['Stadtteil', 'Mean_Inhabitants'],
key_on='feature.properties.Stadtteil',
fill_color='Set2',
fill_opacity=0.75,
line_opacity=0.5,
legend_name='Mean inhabitant over time per district or Stadtteil'
)

folium.LayerControl().add_to(map_pm_3)

display(map_pm_3)

The same plot like above on another map.

In [ ]:
mymap2 = folium.Map(location=cologne_center_geocoord, zoom_start=9,tiles=None)
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(mymap2)

In [ ]:
begin = time.time()


mymap2.choropleth(
 geo_data=choro_pm,
 name='Choropleth',
 data=choro_pm,
 columns=['Bezirk', 'Mean_Inhabitants'],
 key_on='feature.properties.Bezirk',
 fill_color='Reds',
 fill_opacity=1,
 line_opacity=0.2,
 legend_name='Mean Inhabitants over time per district or Bezirk',
 smooth_factor=0
)

display(mymap2)

time.sleep(1)
# store end time
end = time.time()
  
# total time taken
print(f"Total runtime of a folium.choropleth based on a df of shape(2262, 20): is {end - begin}")

Checking again the crs.

In [ ]:
print(choro_pm.geometry[:2])
print(choro_pm.shape)
choro_pm.crs

#### Generating folium.choropleth maps <br> with geopandas groupby dataframes

<p>
The chloropleth above is based on a geopandas data frame of shape (2262, 20).<br>
It is astonishing that geopandas is able to assign the metric to the districts over 2262 rows.<br>
However the calculation requires heavy processing power.<br>
It takes above 10 seconds.
</p>
    
<p>
The calculation cost is diminished, when the data frame is reduced.<br>
This is done below with the groupby-method.<br>
The first step is to select the variables needed for the plot, <br>
then the groupby is applied and the aggregate without calculations is returned with 'first'.
</p>

In [ ]:
mi2244 = choro_pm[['Stadtteil', 'Mean_Inhabitants', 'geometry']].\
groupby(['Stadtteil', 'Mean_Inhabitants']).agg({'geometry': 'first'})

print(mi2244.crs)
mi2244.reset_index(inplace=True)

In [ ]:
# mi2244 = mi2244.set_crs("EPSG:3857")
# EPSG:4326
mi2244 = mi2244.set_crs("EPSG:4326")
print(mi2244.crs)
print(mi2244.geometry.crs)

Assigning a "primary key" to the data frame.<br>
This maybe helpful, but is not necessary here.

In [ ]:
mi2244['PK'] = np.arange(1, mi2244.shape[0] + 1, 1)

In [ ]:
mi2244.columns.to_list()

In [ ]:
mi2244=mi2244[['PK', 'Stadtteil', 'Mean_Inhabitants', 'geometry', None]]

#### What happened? There is suddenly a column "None"!
<p>
Checking the data frame again.<br>
The subsetting has produced a column "None".<br>
There are some clues on Github, why this is the case, but nothing conclusive.<br>
Not only that, but Geopandas determines the "None" <br>
and not the  older "geometry" column as the "geometry" column.<br>
This produces the error:<br>
"TypeError: Object of type Polygon is not JSON serializable"<br>
The older older geometry cannot be transformed as a geometry to Json,<br>
because the "None"-column is allready "JSON serialized".
</p> 

<p>
The strategy to resolve the error here is, <br>
to drop the older geometry column,<br>
rename the Nan to geometry, and then set the geometry column to crs.<br>
It is also possible to change the data type to object or so,<br>
but this produces a warning message.
</p>

In [ ]:
print(type(mi2244))
print(mi2244.shape)
print(mi2244.info())

In [ ]:
print(mi2244.isnull().sum())

In [ ]:
mi2244[:2]

Resolving the "None"- issue.

In [ ]:
mi_5555 = mi2244.drop(columns=['geometry'])

In [ ]:
print(mi_5555.crs)
print(type(mi_5555))
print(mi_5555.info())
mi_5555.head(1)

In [ ]:
mi_6777 = mi_5555.rename(columns={None: 'geometry'})

In [ ]:
mi_6777=mi_6777.set_geometry(col='geometry')

In [ ]:
print(mi_6777.crs)
# mi_6777.to_crs('EPSG:4326')
print(type(mi_6777.geometry))
print(type(mi_6777.geometry[4]))
print(mi_6777.geometry.crs)
mi_6777.head(1)

#### Did the transformation changed the values and do the GeoJson work?

<p>
This questions is answered by comparing the chained data frames.
</p> 
<p>
Data frame: choro_pm
</p> 

In [ ]:
print(choro_pm.crs)
print(type(choro_pm.geometry[2]))
choro_pm.geometry.head(2)

In [ ]:
choro_pm_444_3857 =  choro_pm.to_crs('EPSG:3857')
print(choro_pm_444_3857.crs)
choro_pm_444_3857.head(1)

In [ ]:
choro_pm.to_json()[:500]

<p>
Data frame: sf_1
</p> 

In [ ]:
#  WGS_1984_UTM_Zone_32N
sf_1.geometry = sf_1.geometry.set_crs('EPSG:4326')
sf_1.centroid = sf_1.astype(object)
print(sf_1.columns)
sf_1.info()
sf_1.to_json()[:400]

Data frame: mi_6777

In [ ]:
print(mi_6777.crs)
print(type(mi_6777.geometry[2]))
mi_6777.geometry.head(2)

In [ ]:
mi_6777_js = mi_6777.to_json()

In [ ]:
mi_6777_js[:500]


<p>
The geopspatial data is despite transformations maintained.<br>
The GeoJson's are produced.
</p> 


#### Choropleth map of mean inhabitants by district <br> based on a reduced Geopandas data frame

In [ ]:
# store starting time
begin = time.time()

mi_6777_map = folium.Map(location=cologne_center_geocoord, zoom_start=10)
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(mi_6777_map)


mi_6777_map.choropleth(
 geo_data=mi_6777,
 name='Choropleth',
 data=mi_6777,
 columns=["Stadtteil", 'Mean_Inhabitants'],
 key_on='feature.properties.Stadtteil',
 fill_color='Reds',
 fill_opacity=1,
 line_opacity=0.2,
 legend_name='Districts colored by mean inhabitants over time',
 smooth_factor=0
)

display(mi_6777_map)

time.sleep(1)
# store end time
end = time.time()
  
# total time taken
print(f"Total runtime of the program mean inhabitants is {end - begin}")



The runtime based on the reduced data frame is belwo 2 seconds.

#### Districts colored by population_density

In [ ]:
choro_pm.columns

In [ ]:
choro_pop_density_444 = choro_pm[['Stadtteil', 'population_density', 'geometry']].\
groupby(['Stadtteil', 'population_density']).agg({'geometry': 'first'})
choro_pop_density_444.head()

In [ ]:
print(choro_pop_density_444.crs)


In [ ]:
choro_pop_density_444.reset_index(inplace=True)
print(choro_pop_density_444.info())


In [ ]:
choro_pop_density_444 = choro_pop_density_444.set_crs("EPSG:4326")


In [ ]:
choro_pop_density_444  = choro_pop_density_444.drop(columns=['geometry'])

choro_pop_density_444  = choro_pop_density_444.rename(columns={None: 'geometry'})

choro_pop_density_444 = choro_pop_density_444.set_geometry(col='geometry')

In [ ]:
choro_pop_density_444.info()

In [ ]:
begin = time.time()

choro_pop_density_444_map = folium.Map(location=cologne_center_geocoord, zoom_start=10)
# folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(mi_6777_map)


choro_pop_density_444_map.choropleth(
 geo_data=choro_pop_density_444,
 name='Choropleth',
 data=choro_pop_density_444,
 columns=["Stadtteil", 'population_density'],
 key_on='feature.properties.Stadtteil',
 fill_color='Spectral',
 fill_opacity=1,
 line_opacity=0.2,
 legend_name='Districts colored by population_density',
 smooth_factor=0
)

display(choro_pop_density_444_map)

time.sleep(1)
# store end time
end = time.time()
  
processing_time = round(end - begin,2)
    
# total time taken
print(f"Total runtime of the program \'population_density\' is {processing_time } seconds.")



In [ ]:
choro_pop_density_444['district_centers'] = \
choro_pop_density_444.geometry.to_crs('EPSG:3857').centroid

In [ ]:
choro_pop_density_444['district_centers'] = \
choro_pop_density_444.district_centers.to_crs('EPSG:4326')

In [ ]:
choro_pop_density_444.head(1)

In [ ]:
unvoweled_names_2 = []

for row in choro_pop_density_444.iterrows():
    
    # print(row[1][0])
    unvoweled_name = unidecode.unidecode(row[1][0])
    unvoweled_names_2.append(unvoweled_name)
    
unvoweled_names_2[:5]

In [ ]:
choro_pop_density_444['Stadtteil'] = unvoweled_names_2

In [ ]:
folium.Marker(location=uni_geocoords, popup='Universitaetskliniken:<br>Joseph-Stelzmann-Str. 9',
              icon = folium.Icon(icon='hospital-symbol', prefix='fa',
                                 color='green')).add_to(uniklinik_map)

In [ ]:
for row in choro_pop_density_444.iterrows():
    row_values = row[1]
    district_centers = row_values['district_centers']
    district_location = [district_centers.y, district_centers.x]
    district_popup = ("Stadtteil: " + 
                      str(row_values['Stadtteil']) + 
                      " <br>" + "Population density: <br>" + 
                      str(row_values['population_density']))
    
    marker_districts = \
    folium.Marker(location=district_location, popup = district_popup,
                  icon = folium.Icon(icon='anchor', prefix='fa',
                                 color='red'))
    marker_districts.add_to(choro_pop_density_444_map)
    
display(choro_pop_density_444_map)
    